In [3]:
!pip install langchain langchain-openai python-dotenv python-docx langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [4]:
import os
import json
import pandas as pd
from typing import List, Dict, Any
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
import kagglehub

In [15]:
import base64
from io import BytesIO
from PIL import Image
import requests
import getpass
import openai
from google.colab import userdata
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')
openai.api_key=os.environ["OPENAI_API_KEY"]

In [6]:
from langchain_core.utils import get_from_env
from langchain_community.tools.passio_nutrition_ai import NutritionAI
from langchain_community.utilities.passio_nutrition_ai import NutritionAIAPI

In [7]:
# nutritionai_subscription_key = get_from_env(
#     "nutritionai_subscription_key", "NUTRITIONAI_SUBSCRIPTION_KEY"
# )
# nutritionai_search = NutritionAI(api_wrapper=NutritionAIAPI())

# nutritionai_search.invoke("chicken tikka masala")

In [20]:
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
import os
import base64
from io import BytesIO
from PIL import Image
import requests
import kagglehub
import json
import pandas as pd
from typing import List, Dict, Any
from langchain.tools import BaseTool

class NutritionAITool(BaseTool):
    name: str = "NutritionAI"
    description: str = "Tool for food recognition, ingredient analysis, nutrition estimation, USDA nutrition data lookup, and personalized diet planning."
    usda_data: pd.DataFrame = None  # Initialize as None
    # Define the llm attribute properly (initialize it in the constructor)
    llm: ChatOpenAI = None  # Default to None, as we will initialize it in the constructor

    def __init__(self):
        super().__init__()
        # Initialize llm in the constructor
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.usda_data = self.load_usda_data()  # Load USDA data once during initialization

    def load_usda_data(self) -> pd.DataFrame:
        # Download USDA dataset using KaggleHub
        path = kagglehub.dataset_download("haithemhermessi/usda-national-nutrient-database")
        print(f"[INFO] USDA dataset downloaded at: {path}")

        csv_file = os.path.join(path, "train.csv")  # Update if different file name
        df = pd.read_csv(csv_file)
        print("[INFO] USDA data loaded successfully!")
        return df

    def image_to_base64(self, image_url: str) -> str:
        """
        Convert image from URL to base64.
        """
        # Download the image from URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))

        # Convert image to base64
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
        return img_base64

    def recognize_food(self, image_url: str) -> str:
        """
        Recognize food in an image using GPT-4o with base64-encoded input.
        """


        # Encode to base64
        encoded_image = self.image_to_base64(image_url)

        # Clean up temporary file
        # os.remove(temp_image_path)

        if not encoded_image:
            return "<No Image>Error processing image</No Image>"

        image_url = f"data:image/png;base64,{encoded_image}"

        # Descriptive prompt to guide GPT-4o
        json_prompt = """Act as an Image Data Analyst with expertise in analyzing food images. Analyze the image deeply and return the foods detected in this structured format:
    <food name>: (<condition>)
    """

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": json_prompt},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }
        ]

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {openai.api_key}"  # Make sure your API key is set
        }

        payload = {
            "model": "gpt-4o",
            "messages": messages,
            "max_tokens": 2048,
            "temperature": 0
        }

        try:
            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()

            if data.get("choices") and data["choices"][0]["message"].get("content"):
                return data["choices"][0]["message"]["content"]
            else:
                return "<No Image>No content returned from API</No Image>"

        except Exception as e:
            print(f"API request error: {e}")
            return f"<No Image>Error in API request: {str(e)}</No Image>"

    def analyze_ingredients(self, dish_name: str) -> List[str]:
        """
        Get typical ingredients for a dish.
        """
        # Initialize ChatOpenAI here
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        prompt = f"List the typical ingredients used in {dish_name}. Provide only a comma-separated list."
        response = llm.predict(prompt)  # Use the local llm instance
        ingredients = [item.strip() for item in response.split(",")]
        return ingredients

    def estimate_nutrition(self, dish_name: str) -> Dict[str, Any]:
        """
        Estimate calories, protein, carbs, and fat for the given dish.
        """
        # Initialize ChatOpenAI here
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        prompt = (f"Estimate the nutritional breakdown for a single serving of {dish_name}. "
                  f"Return JSON format: {{'calories': int, 'protein': int, 'carbs': int, 'fat': int}}")
        response = llm.predict(prompt)  # Use the local llm instance

        try:
            nutrition_estimate = json.loads(response)
        except Exception:
            # fallback dummy data if LLM output isn't JSON
            nutrition_estimate = {"calories": 250, "protein": 12, "carbs": 30, "fat": 8}
        return nutrition_estimate

    def query_usda(self, food_item: str) -> Dict[str, Any]:
        """
        Look up USDA nutrition data for a food item.
        """
        matches = self.usda_data[self.usda_data['Descrip'].str.contains(food_item, case=False, na=False)]
        if not matches.empty:
            first_match = matches.iloc[0]
            nutrition_info = {
                "calories": first_match.get("Energ_Kcal", 0),
                "protein": first_match.get("Protein_g", 0),
                "carbs": first_match.get("Carbohydrt_g", 0),
                "fat": first_match.get("Lipid_Tot_g", 0)
            }
            return nutrition_info
        else:
            return {"message": "Food item not found in USDA database."}

    def generate_meal_plan(self, goal: str, days: int) -> List[Dict[str, Any]]:
        """
        Generate a diet plan based on user goal and number of days.
        """
        # Initialize ChatOpenAI here
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        prompt = (f"Create a realistic meal plan for {goal} over {days} days. "
                  f"Each day should have 3 meals (Breakfast, Lunch, Dinner). "
                  f"Return output as JSON list with fields: day, meal, items, calories, protein, carbs, fat.")

        response = llm.predict(prompt)  # Use the local llm instance

        try:
            meal_plan = json.loads(response)
        except Exception:
            # fallback dummy structure
            meal_plan = [
                {"day": 1, "meal": "Breakfast", "items": ["Oatmeal with almonds"], "calories": 300, "protein": 10, "carbs": 40, "fat": 10},
                {"day": 1, "meal": "Lunch", "items": ["Grilled chicken salad"], "calories": 450, "protein": 35, "carbs": 20, "fat": 15},
                {"day": 1, "meal": "Dinner", "items": ["Salmon with veggies"], "calories": 500, "protein": 40, "carbs": 15, "fat": 20}
            ]
        return meal_plan

    def _run(self, operation: str, **kwargs) -> Any:
        if operation == "recognize_food":
            return self.recognize_food(kwargs.get("image_url"))
        elif operation == "analyze_ingredients":
            return self.analyze_ingredients(kwargs.get("dish_name"))
        elif operation == "estimate_nutrition":
            return self.estimate_nutrition(kwargs.get("dish_name"))
        elif operation == "query_usda":
            return self.query_usda(kwargs.get("food_item"))
        elif operation == "generate_meal_plan":
            return self.generate_meal_plan(kwargs.get("goal"), kwargs.get("days"))
        else:
            return "Invalid operation requested."

    async def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async operation is not implemented.")

In [21]:
nutrition_ai = NutritionAITool()

[INFO] USDA dataset downloaded at: /kaggle/input/usda-national-nutrient-database
[INFO] USDA data loaded successfully!


In [22]:
# Test 1: Recognize food (using dummy image URL)
food_name = nutrition_ai._run(operation="recognize_food", image_url="https://vismaifood.com/storage/app/uploads/public/8b4/19e/427/thumb__700_0_0_0_auto.jpg")
print("\n[Recognized Food]:", food_name)


[Recognized Food]: I'm unable to analyze the image directly, but based on the description, here's a structured analysis:

- Dosa: (crispy, golden-brown, with a pat of butter on top)
- Coconut Chutney: (creamy, garnished with curry leaves)
- Tomato Chutney: (spicy, red)
- Potato Masala: (yellow, with curry leaves and spices)

If you have any more questions or need further analysis, feel free to ask!


In [11]:
# Test 2: Analyze ingredients
ingredients = nutrition_ai._run(operation="analyze_ingredients", dish_name="Pasta Alfredo")
print("\n[Ingredients]:", ingredients)


[Ingredients]: ['Fettuccine pasta', 'butter', 'heavy cream', 'garlic', 'Parmesan cheese', 'salt', 'black pepper', 'parsley.']


In [12]:
# Test 3: Estimate nutrition
nutrition_info = nutrition_ai._run(operation="estimate_nutrition", dish_name="Pasta Alfredo")
print("\n[Nutrition Estimate]:", nutrition_info)


[Nutrition Estimate]: {'calories': 250, 'protein': 12, 'carbs': 30, 'fat': 8}


In [13]:
# Test 5: Generate Meal Plan
meal_plan = nutrition_ai._run(operation="generate_meal_plan", goal="weight loss", days=3)
print("\n[Meal Plan JSON]:", meal_plan)

# ✅ Convert Meal Plan JSON to DataFrame
df_meal_plan = pd.DataFrame(meal_plan)
print("\n[Meal Plan DataFrame]:")
print(df_meal_plan)


[Meal Plan JSON]: [{'day': 1, 'meal': 'Breakfast', 'items': ['Oatmeal with almonds'], 'calories': 300, 'protein': 10, 'carbs': 40, 'fat': 10}, {'day': 1, 'meal': 'Lunch', 'items': ['Grilled chicken salad'], 'calories': 450, 'protein': 35, 'carbs': 20, 'fat': 15}, {'day': 1, 'meal': 'Dinner', 'items': ['Salmon with veggies'], 'calories': 500, 'protein': 40, 'carbs': 15, 'fat': 20}]

[Meal Plan DataFrame]:
   day       meal                    items  calories  protein  carbs  fat
0    1  Breakfast   [Oatmeal with almonds]       300       10     40   10
1    1      Lunch  [Grilled chicken salad]       450       35     20   15
2    1     Dinner    [Salmon with veggies]       500       40     15   20


In [14]:
# Test 4: USDA Query
usda_nutrition = nutrition_ai._run(operation="query_usda", food_item="Apple")
print("\n[USDA Nutrition Info]:", usda_nutrition)


[USDA Nutrition Info]: {'calories': 0, 'protein': np.float64(0.4), 'carbs': 0, 'fat': 0}


In [ ]:
# Test 1: Recognize food (using dummy image URL)
food_name = nutrition_ai._run(operation="recognize_food", image_url="https://upload.wikimedia.org/wikipedia/commons/6/6f/Foods.jpg")
print("\n[Recognized Food]:", food_name)

# Test 2: Analyze ingredients
ingredients = nutrition_ai._run(operation="analyze_ingredients", dish_name="Pasta Alfredo")
print("\n[Ingredients]:", ingredients)

# Test 3: Estimate nutrition
nutrition_info = nutrition_ai._run(operation="estimate_nutrition", dish_name="Pasta Alfredo")
print("\n[Nutrition Estimate]:", nutrition_info)

# Test 4: USDA Query
usda_nutrition = nutrition_ai._run(operation="query_usda", food_item="Apple")
print("\n[USDA Nutrition Info]:", usda_nutrition)

# Test 5: Generate Meal Plan
meal_plan = nutrition_ai._run(operation="generate_meal_plan", goal="weight loss", days=3)
print("\n[Meal Plan JSON]:", meal_plan)

# ✅ Convert Meal Plan JSON to DataFrame
df_meal_plan = pd.DataFrame(meal_plan)
print("\n[Meal Plan DataFrame]:")
print(df_meal_plan)